# Feature Extraction

Models (with tuned hyperparameters) are compared on a classification problem. Final extracted features are computed.

## Load pre-computed features
The six outputs of preprocessing are imported:
1. 'data/prepped.pkl' is a pickled dataframe containing cleaned comments and their subreddit label
2. 'data/feature_names.pkl' is a pickled series containing the words that were counted
3. 'data/term_counts.npz' is a scipy sparse matrix containing the word counts
4. 'data/tf.npz' is a scipy sparse matrix containing the word frequencies
5. 'data/tf.npz' is a scipy sparse matrix containing the word frequencies weighted by their inverse document frequency
6. 'data/gensim_sentences.pkl' is a list of gensim labelled sentence objects

In [1]:
import numpy as np
import pandas as pd
from time import time
from scipy.sparse import load_npz
import pickle
from gensim.utils import unpickle as gensim_unpickle

print("Importing data...")
t0 = time()
data = pd.read_pickle('data/prepped.pkl')
with open('data/feature_names.pkl', 'rb') as fp:
    feature_names = pickle.load(fp)
cts = load_npz('data/term_counts.npz')
tf = load_npz('data/tf.npz')
tfidf = load_npz('data/tfidf.npz')
sentences = gensim_unpickle('data/gensim_sentences.pkl')
print("done in %0.3fs" % (time()-t0))


C:\Users\sfzba\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Importing data...
done in 37.981s


## Classification
In order to compare feature extraction methods using a gold standard, we formulate a classification problem against each comments' subreddit.

### Cross-validation
Twenty percent of the data is left out for cross-validation, with class balances kept intact as best as possible.

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# X = data['body'].astype('unicode')
# X = cts
# X = tf
X = tfidf

le = LabelEncoder()
y = le.fit_transform(data.subreddit)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=True, stratify=y)
sents_train, sents_test = train_test_split(sentences, random_state=42, test_size=0.2, shuffle=True, stratify=y)


### Compute word frequency matrix decompositions on training data

#### LSA

In [3]:
from sklearn.decomposition import TruncatedSVD

# Global hyper-parameter
n_components = 60

print("Fitting LSA model")
t0 = time()
LSA_model = TruncatedSVD(n_components=n_components, random_state=0).fit(X_train)
LSA_train = LSA_model.transform(X_train)
LSA_test  = LSA_model.transform(X_test)
print("done in %0.3fs." % (time() - t0))
print()


Fitting LSA model
done in 67.320s.



#### pLSA

In [5]:
from sklearn.decomposition import NMF

print("Fitting pLSA model")
# Use the non-negative matrix factorization with generalized Kullback-Leibler divergence
t0 = time()
pLSA_model = NMF(n_components=n_components, random_state=0,
                 beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
                 l1_ratio=.5).fit(X_train)
pLSA_train = pLSA_model.transform(X_train)
pLSA_test  = pLSA_model.transform(X_test)
print("done in %0.3fs." % (time() - t0))
print()


Fitting pLSA model


KeyboardInterrupt: 

#### LDA

In [6]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

print("Fitting LDA model")
t0 = time()
LDA_model = LatentDirichletAllocation(n_components=n_components, learning_decay=0.51, learning_offset=2,
                                      learning_method='online', random_state=0).fit(X_train)
LDA_train = LDA_model.transform(X_train)
LDA_test  = LDA_model.transform(X_test)
print("done in %0.3fs." % (time() - t0))
print()


Fitting LDA model


KeyboardInterrupt: 

#### doc2vec

In [ ]:
from gensim.models import Doc2Vec

print("Training doc2vec")
model = Doc2Vec(size=n_components, iter=75, dm=1, min_count=2)
model.build_vocab(sents_train)
model.train(sents_train, total_examples=model.corpus_count, epochs=model.iter)

doc2vec_train = np.empty([model.docvecs.count, n_components])
for i in range(model.docvecs.count):
    doc2vec_train[i,:] = model.docvecs[i]

doc2vec_test = np.empty([len(sents_test), n_components])
for i, vec in enumerate(sents_test):
    doc2vec_test[i,:] = model.infer_vector(vec.words)

print("done in %0.3fs." % (time() - t0))
print()


#### Save outputs to disk in case of computation interruption

In [7]:
# extractors = [
#     ("LSA", LSA_train, LSA_test),
#     ("pLSA", pLSA_train, pLSA_test),
#     ("LDA", LDA_train, LDA_test),
#     ("doc2vec", doc2vec_train, doc2vec_test),
# ]
# with open('data/temp_extractors.pkl', 'wb') as fp:
#     pickle.dump(extractors, fp)

with open('data/temp_extractors.pkl', 'rb') as fp:
    extractors = pickle.load(fp)
# _, LSA_train, LSA_test = extractors[0]
# _, pLSA_train, pLSA_test = extractors[1]
# _, LDA_train, LDA_test = extractors[2]
# _, doc2vec_train, doc2vec_test = extractors[3]

In [22]:
extractors[3][2].shape

(362517, 10)

In [21]:
len(sents_test)

362517

## Test feature extractors using various classifiers

KNN is directly related to the method for subreddit recommendation. It is therefore a reasonable choice for classifier here.

If desired, it may interesting to compare to fast, memory-efficient methods like SGD implementations of a linear SVM and logisitic regression, or a simple decision tree.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier

extractors = [
     ("LSA", LSA_train, LSA_test),
     ("pLSA", pLSA_train, pLSA_test),
     ("LDA", LDA_train, LDA_test),
     ("doc2vec", doc2vec_train, doc2vec_test),
]

classifiers = [
#    ("Linear SVM", SGDClassifier(loss='hinge', n_jobs=-1, random_state=42)),
#    ("Logistic Regression", SGDClassifier(loss='log', n_jobs=-1, random_state=42)),
    ("Nearest Neighbors", KNeighborsClassifier()),
#    ("Decision Tree", DecisionTreeClassifier(max_depth=5)),
]

for ex_name, ex_train, ex_test in extractors:
    print("%s" % ex_name)
    print()
    for cl_name, clf in classifiers:
        print("   %s" % cl_name)
        print()
        
        # F1 score undefined for any label with no predicted members
        # Use accuracy instead
        t0 = time()
        clf.fit(ex_train, y_train)
        t1 = time()
        score = clf.score(ex_test, y_test) 
        t2 = time()
        train_score = clf.score(ex_train, y_train)
        
        print("           Train")
        print("   Train time: %0.3f sec" % (t1-t0))
        print("   Accuracy  : %0.2f %%" % (100*train_score))
        print("   Acc. mult.: %0.2f" % (train_score*(y.max()-y.min()+1)))
        print("           Test")
        print("   Score time: %0.3f sec" % (t2-t1))
        print("   Accuracy  : %0.2f %%" % (100*score))
        print("   Acc. mult.: %0.2f" % (score*(y.max()-y.min()+1)))
        print()
    

In [ ]:
does not compile

## Compute decompositions at subreddit level

In [ ]:
import numpy as np
import pandas as pd
from time import time
from scipy.sparse import load_npz
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from gensim.models import Doc2Vec
from gensim.utils import unpickle as gensim_unpickle

# cts = load_npz('data/term_counts_sub.npz')
# tf = load_npz('data/tf_sub.npz')
tfidf = load_npz('data/tfidf_sub.npz')
sentences = gensim_unpickle('data/gensim_sentences_sub.pkl')

X = tfidf

n_components = 60

print("LSA...")
t0 = time()
LSA = TruncatedSVD(n_components=n_components, random_state=0).fit(X)
X_LSA = LSA.transform(X)
np.save('data/X_LSA_sub.npy', X_LSA)
print("done in %0.3fs." % (time() - t0))
print()

print("pLSA...")
t0 = time()
pLSA = NMF(n_components=n_components, random_state=0,
           beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
           l1_ratio=.5).fit(X)
X_pLSA = pLSA.transform(X)
np.save('data/X_pLSA_sub.npy', X_pLSA)
print("done in %0.3fs." % (time() - t0))
print()

print("LDA...")
t0 = time()
LDA = LatentDirichletAllocation(n_components=n_components, learning_decay=0.51, learning_offset=2,
                                learning_method='online', random_state=0).fit(X)
X_LDA = LDA.transform(X)
np.save('data/X_LDA_sub.npy', X_LDA)
print("done in %0.3fs." % (time() - t0))
print()

print("doc2vec...")
t0 = time()
np.random.shuffle(sentences) # may improve accuracy
model = Doc2Vec(size=n_components, iter=75, dm=1, min_count=2)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
# Form numpy matrix from gensim array
doc2vec = np.empty([model.docvecs.count, n_components])
for i in range(model.docvecs.count):
    doc2vec[i,:] = model.docvecs[i]
np.save('data/doc2vec_sub.npy', doc2vec)
print("done in %0.3fs." % (time() - t0))
print()


In [ ]:
with open('data/extraction_models.pkl', 'rb') as fp:
    pickle.dump((LSA, pLSA, LDA, doc2vec), fp)

## Visualizations for word frequency matrix decompositions
Each column of the output matrices can be seen as a topic, some weighted mixture of words. The most heavily weighted words for each topic are shown, either as a text list, a plot of their respective weights, or as a wordcloud (with size proportional to each words' weight).

In [ ]:
X = load_npz('data/tfidf.npz')
print("LSA...")
t0 = time()
LSA = TruncatedSVD(n_components=n_components, random_state=0).fit(X)
print("done in %0.3fs." % (time() - t0))


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import pickle
with open('data/feature_names.pkl', 'rb') as fp:
    feature_names = pickle.load(fp)
# with open('data/extraction_models.pkl', 'rb') as fp:
#     LSA, pLSA, LDA, doc2vec = pickle.load(fp)
n_top_words = 10

fs = 18

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

def plot_top_words(model, feature_names, n_top_words):
    #norm_comps = model.components_ / model.components_.sum(axis=1)[:, np.newaxis]
    for topic_idx, topic in enumerate(model.components_):
        x = range(n_top_words,0,-1)
        sort_idx = topic.argsort()[:-n_top_words - 1:-1]
        plt.figure(figsize=[6,6])
        plt.barh(x, topic[sort_idx])
        plt.yticks(x, np.asarray(feature_names)[sort_idx], fontsize=fs)
        plt.title(("Topic #%d: " % topic_idx), fontsize=fs)
        plt.xlabel("LSA Weight", fontsize=fs)
        plt.tight_layout()
        plt.show()

def show_top_wc(model, feature_names, n_top_words):
    wc = WordCloud(background_color="white", max_words=n_top_words)
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d: " % topic_idx)
        words = {}
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            words[feature_names[i]] = topic[i]
        wc.generate_from_frequencies(words)
        plt.imshow(wc, interpolation="bilinear")
        plt.axis("off")
        plt.tight_layout()
        plt.show()

print("\nTopics in LSA model:")
# print_top_words(LSA, feature_names, n_top_words)
plot_top_words(LSA, feature_names, n_top_words)

# print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
# # print_top_words(pLSA, feature_names, n_top_words)
# plot_top_words(pLSA, feature_names, n_top_words)
# 
# print("\nTopics in LDA model:")
# # print_top_words(LDA, feature_names, n_top_words)
# plot_top_words(LDA, feature_names, n_top_words)
